In [3]:
library(MASS)

# Create data X and label y

The data set X: please read simulation_AR (pdf file) on how the construction of X.
X (T*p) satisfies the following:
* Each row of X has covariance matrix cov_feature, which has three block matrix and an identity matrix on the diagnal.
* Each column of X has the same longitudinal covariance structure (AR).

The label y: suppose there are 100*4 features, denoted by $X^{(1)},X^{(2)},...,X^{(400)}$.
Our model is $$y_{t} = 5X^{(1)}_{t}+2X^{(2)}_{t}+2X^{(3)}_{t}+5X^{(2)}_{t}X^{(3)}_{t}
+5X_{t}^{(301)}+2X^{(302)}_{t}+2X^{(303)}_{t}+5X^{(302)}_{t}X^{(303)}_{t}
+\epsilon_{t}$$ where $\epsilon_{t}\stackrel{}{\sim} N(0,\alpha I)$. $\alpha$ is the noise level (in the code, it is called var_noise)

Therefore, the expected results of feature selection are:
* The module 1 and 4 are chosen
* The feature 1,2,3,301,302,303 are chosen

In [51]:
# p features and T oberservations
# p= 4*p0 : the first 3 p0 features are 3 modules, independent bewteen modules and
# correlated (cov=0.8) within each. The last module is independent within and with the 
# first three

# return a T*(p+1) matrix
Data_CS =function(T,var_noise) {
    p0 = 100
    p = 4*p0
    
    #### covariance matrix between features: it is either 0 (independent) or 0.8 ####
    cov_feature = matrix(0,nrow = p, ncol = p)
    # cov within the first three modules
    cov_star = matrix(0.8,nrow = p0,ncol = p0)
    diag(cov_star)=1
    # put cov_star into cov_feature
    cov_feature[1:p0,1:p0] = cov_star
    cov_feature[(p0+1):(2*p0),(p0+1):(2*p0)] = cov_star
    cov_feature[(2*p0+1):(3*p0),(2*p0+1):(3*p0)] = cov_star
    cov_feature[(3*p0+1):(4*p0),(3*p0+1):(4*p0)] = diag(p0)
    ####
    
    #### simulating data ####
    # T*p Data matrix, pre-allocate memory
    X_data = matrix(0,nrow = T,ncol = p)
    # generate gamma 
    gamma = mvrnorm(n=1,rep(0,p),cov_feature)
    #generate each row
    for (i in 1:T){
        X_data[i,] = 0.8*gamma+0.6*mvrnorm(n=1,rep(0,p),cov_feature)
        }
    ###
    
    ### create labels ###
    # create a n vector for labels
    y = matrix(data=0,nrow = T)
    # build y according to our model
    y = (5*X_data[,1]+2*X_data[,2]+2*X_data[,3]+5*X_data[,2]*X_data[,3]
         +5*X_data[,301]+2*X_data[,302]+2*X_data[,303]+5*X_data[,302]*X_data[,303]
         +mvrnorm(n = 1, rep(0, T), diag(x=var_noise,T)))
    ###
    
    return (cbind(X_data,y))

}

In [59]:
set.seed(0)
Data_CS(T=5,var_noise = 0.3)

,,,,,,,,,,...,,,,,,,,,,y
1.7407866,0.7165797,1.3688117,1.0314631,1.3876805,1.4874913,1.6287665,1.5960559,1.8400291,0.4229733,...,-0.5573129,0.9554073,2.754275,0.06523443,0.81441678,-0.9149480,-2.345674,1.30168895,1.2937165,20.227139
1.5324829,1.0467967,1.6236058,0.8724605,1.2173783,1.3320388,2.0153044,1.6049079,1.5313411,0.9952753,...,-0.5569945,1.3555163,1.646501,0.70833466,0.08101691,-0.4351898,-1.912799,0.02901912,0.4661387,17.033645
1.5093328,0.6792006,1.3579350,0.4884840,1.1224566,1.6851864,1.2960038,0.9839399,1.6907128,1.2139573,...,-0.2602909,0.7786797,2.246425,-0.27743072,-0.02982492,-0.1044265,-2.615714,0.78831001,1.0831711,8.790030
-0.3041881,-1.9530938,-1.1427849,-1.2343919,-1.3680662,-0.8568481,-1.0319428,-0.6757070,-0.8279082,-1.3904139,...,-0.2005170,0.8967164,1.759592,0.09423212,-0.47172281,-0.9600292,-0.966871,-0.16834060,0.9950211,-7.015299
0.5991773,-0.1840922,0.4274708,-0.2199940,0.4973094,0.3406958,0.3146909,0.9434247,0.8198816,0.3843253,...,-1.4968179,0.1176065,1.418490,0.09473808,-0.06373402,-1.1522195,-1.593592,0.62712616,0.8365169,-4.308122
